In [1]:
import os
import sys
os.environ['PATH']+=':'+sys.path[1].split('lib')[0]+'bin'
sys.path.append('/home/diego/Projects')

import tqdm
import tempfile
import numpy as np
import matplotlib.pyplot as plt

import simtk.openmm as openmm
import simtk.unit as unit
import simtk.openmm.app as app
from openmmtools import states, mcmc
from yank.multistate import ParallelTemperingSampler, MultiStateReporter, ParallelTemperingAnalyzer

import TestSystems.testsystems_uibcdf as TestSystems

plt.style.use("ggplot")

# Setting up the system

In [2]:
working_system = TestSystems.CB7B2Implicit()
positions = working_system.positions

In [3]:
working_system.make_nglview()

NGLWidget()

# Setting up the experiment variables

In [5]:
experiment_continuation= False
minimize = True # minimize
equilibrate = True # equilibrate

# minimization
tolerance = 1.0 * unit.kilocalories_per_mole / unit.nanometer
maximum_evaluations = 5000

# equilibration
collision_rate = 5.0 / unit.picosecond #Langevin
timestep_equilibration = 1.0 * unit.femtoseconds
nequiliterations = 100
nsteps_equilibration = 10000

# Alchemy
lambda_electrostatics = [1.00, 0.95, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40,
                         0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00,
                         0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
lambda_sterics        = [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00,
                         1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60,
                         0.50, 0.40, 0.30, 0.20, 0.15, 0.10, 0.05, 0.00]
# Replica Exchange
restraint = 'Harmonic'
timestep = 2.0 * unit.femtoseconds # timestep for simulation   
nsteps = 50 # number of timesteps per iteration (exchange attempt)
niterations = 1000 # number of iterations